# Basic Alhazen Agent / Chatbot.

> Building a simple application to allow the user to chat to Alhazen. 

In [ ]:
#| default_exp agent

In [ ]:
#| export

from alhazen.core import PromptTemplateRegistry, load_alhazen_tool_environment
from alhazen.schema_sqla import *
from alhazen.tools.basic import IntrospectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import * 
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, AgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.output_parsers.json import parse_json_markdown
from langchain.pydantic_v1 import BaseModel
from langchain.tools.render import render_text_description, render_text_description_and_args
from langchain.agents.output_parsers.json import JSONAgentOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool


from langchain.prompts import ChatPromptTemplate

import os
import pandas as pd
from pathlib import Path
import re
import requests
import sys
from typing import Dict, List, Optional, Sequence, Union, Any
from uuid import UUID

/Users/gully.burns/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export 

class AlhazenAgent:
    '''The base Alhazen agent.'''

    def __init__(self, agent_llm, tool_llm, db_name, sml=None, verbose=True, return_intermediate_steps=True):

        loc = load_alhazen_tool_environment()

        self.db = Ceifns_LiteratureDb(loc=loc, name=db_name)
        self.agent_llm  = agent_llm
        self.tool_llm  = tool_llm

        self.tk = AlhazenToolkit(db=self.db, llm=self.tool_llm)
        self.tools = self.tk.get_tools()

        pts = PromptTemplateRegistry()
        pts.load_prompts_from_yaml('alhazen_base.yaml')

        self.prompt = pts.get_prompt_template('structured chat agent').generate_chat_prompt_template()
        
        agent = create_structured_chat_agent_withFixes(self.agent_llm, self.tools, self.prompt)
 
        self.agent_executor = AgentExecutor(agent=agent, 
                                            tools=self.tools,
                                            verbose=verbose, 
                                            return_intermediate_steps=return_intermediate_steps)

def create_structured_chat_agent_withFixes(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """modified from `create_structured_chat_agent` to deal with escaped underscore characters."""

    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
        prompt.input_variables
    )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=render_text_description_and_args(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm_with_stop = llm.bind(stop=["Observation"])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | JSONAgentOutputParser_withFixes()
    )
    return agent

class JSONAgentOutputParser_withFixes(AgentOutputParser):
    """modified from `JSONAgentOutputParser` to deal with escaped underscore characters."""

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        try:
            text = text.strip()

            # Hack to remove escaped underscores.
            text = re.sub('\\\\_', '_', text)
            text = re.sub('\\\\_', '_', text)

            # Hack to look for the JSON code fragment 
            # inside other text inside the 
            m = re.search('.*?([\[\{](.|\n)*[\}\]]).*?', text, flags=re.M)
            if m:
                text = m.group(1)

            response = parse_json_markdown(text)
            if isinstance(response, list):
                # gpt turbo frequently ignores the directive to emit a single action
                response = response[0]
            if response["action"] == "Final Answer":
                return AgentFinish({"output": response["action_input"]}, text)
            else:
                return AgentAction(
                    response["action"], response.get("action_input", {}), text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e

    @property
    def _type(self) -> str:
        return "json-agent"
